# Постановка Задачи
### Validation

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

# Решение

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [2]:
# на будущее запомнить, если гугл из-за вирусов просит подтверждения, 
# то использовать данную конструкцию !gdown "<drive-id>&confirm=t"
!gdown "1nU4F3bYJK2JcMwZuDCpoENE8iVZQxQbl&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1nU4F3bYJK2JcMwZuDCpoENE8iVZQxQbl&confirm=t
To: /content/likes_data.zip
100% 362M/362M [00:03<00:00, 97.0MB/s]


In [3]:
!unzip likes_data.zip

Archive:  likes_data.zip
replace test? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test                    
  inflating: __MACOSX/._test         
  inflating: track_artists.csv       
  inflating: train                   
  inflating: __MACOSX/._train        


## Формирование датасета в нужном виде

In [4]:
data_list = []

with open('/content/train', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        tracks = [int(n) for n in line.split()]
        user_tracks = np.empty((len(tracks), 3), dtype=int)
        user_tracks[:, 0] = i # user_id
        user_tracks[:, 1] = tracks[::-1] # track_id
        user_tracks[:, 2] = np.arange(len(tracks)) # order of track_id 
        data_list.append(user_tracks)

data_arr = np.vstack(data_list)

In [5]:
df = pd.DataFrame(data_arr, columns = ['user_id', 'track_id', 'order'])

Для упрощения расчетов ограничемся только первыми 1000 пользователями

In [6]:
df = df[df['user_id'].isin(np.arange(1000))]
print(df.shape)
df.head()

(79953, 3)


,user_id,track_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4


Проверим

In [7]:
df['user_id'].nunique()

1000

## Реализация валидации

In [8]:
class UsersKFold():
    def __init__(self, n_folds: int, p: int, random_seed: int=42):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df: pd.DataFrame):
        df = df.copy()
        users = df['user_id'].unique()
        # Разбивка по фолдам
        users_kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)
        
        for train_users, test_users in users_kfold.split(users):
            # Получение масок
            train_mask = df['user_id'].isin(train_users)
            test_mask = df['user_id'].isin(test_users) & (df['order'] < self.p)
            yield train_mask, test_mask

Проверка

In [9]:
n_folds = 4
p = 5

cv = UsersKFold(n_folds=n_folds, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
    train_fold = df[train_mask]
    test_fold = df[test_mask]

    if (np.in1d(train_fold['user_id'].unique(), test_fold['user_id'].unique())).sum() == 0:
      print(f'Фолд= {i}, Нет общих пользователей')
    else:
      print('Ошибка, есть общие пользователи')
    
    if test_fold.groupby('user_id').count().values.max() <= p:
      print(f'Фолд= {i}, в тест выборке меньше {p} последних треков')
    else:
      print(f'Ошибка, в тест выборке больше {p} последних треков')
    print('*****'*10)

Фолд= 0, Нет общих пользователей
Фолд= 0, в тест выборке меньше 5 последних треков
**************************************************
Фолд= 1, Нет общих пользователей
Фолд= 1, в тест выборке меньше 5 последних треков
**************************************************
Фолд= 2, Нет общих пользователей
Фолд= 2, в тест выборке меньше 5 последних треков
**************************************************
Фолд= 3, Нет общих пользователей
Фолд= 3, в тест выборке меньше 5 последних треков
**************************************************
Фолд= 4, Нет общих пользователей
Фолд= 4, в тест выборке меньше 5 последних треков
**************************************************
